In [1]:
import os
import bs4 as bs
import requests
import pandas as pd
from recsys.imdb_parser import metadata
from recsys.imdb_parser import identifiers
from recsys import utils

%load_ext autoreload
%autoreload 2

In [17]:
response = requests.get('https://www.imdb.com/title/tt0468569/', headers={"Accept-Language": "en-US,en;q=0.5"})
soup = bs.BeautifulSoup(response.text, 'lxml')

In [50]:
CREDENTIALS_PATH = os.path.join('..', 'scripts', 'config', 'credentials.yaml')

aws_credentials = utils.parse_config(CREDENTIALS_PATH, 'aws')
access_key = aws_credentials['access_key']
secret_access_key = aws_credentials['secret_access_key']

In [5]:
df = pd.DataFrame(data={'a': [1, 2, 3], 'b': [1, 4, 6]})
df

,a,b
0,1,1
1,2,4
2,3,6


In [ ]:
df

In [56]:
utils.save_df(df, 'test.csv', metadata_config, aws_credentials)

SaveResponse(is_successful=True, exception_msg='')

In [39]:
soup.find('li', {'data-testid': 'title-techspec_runtime'}).div.text

'2 hours 32 minutes'

In [12]:
CONFIG_FILE = os.path.join('..', 'scripts', 'config', 'parser_config.yaml')
metadata_config = utils.parse_config(CONFIG_FILE, 'metadata')
metadata_config

{'metadata_file': 'data/metadata/metadata.json',
 's3_bucket': 'movie-on-friday',
 's3_poster_dir': 'posters',
 'dynamodb_region': 'us-east-1',
 'genres': 'all',
 'min_delay': 0.1,
 'max_delay': 0.2,
 'log_file': 'logs/imdb_parser/details.log',
 'log_level': 'INFO',
 'log_msg_format': '%(asctime)s %(levelname)s %(message)s',
 'log_dt_format': '%Y-%m-%d %H:%M:%S'}

In [6]:
movie_metadata = utils.read_json(os.path.join('..', metadata_config['metadata_file']))
genres = {item['main_genre'] for item in movie_metadata.values()}
genres

{'Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Western'}

In [24]:
import sys

sys.getsizeof(movie_metadata) / 1024 / 1024

0.140716552734375

In [26]:
sys.getsizeof(batman_metadata) * 2600 / 1024 / 1024

0.89263916015625

In [56]:
detcol = metadata.MetadataCollector(metadata_config, aws_credentials)

In [57]:
detcol.collect()

Animation              0%|                    | 0/1 [00:00<?, ?it/s]

Western              100%|████████████████████| 1/1 [00:03<00:00,  3.38s/it]


'tt0892769'

In [41]:
batman_metadata = detcol.collect_title_details(soup)

In [20]:
batman_metadata.keys()

dict_keys(['original_title', 'genres', 'poster', 'review_summary', 'agg_rating', 'actors', 'imdb_recommendations', 'details', 'boxoffice', 'runtime'])

In [43]:
batman_metadata['details']

{'release_date': ['July 18, 2008 (United States)'],
 'countries_of_origin': ['United States', 'United Kingdom'],
 'language': ['English', 'Mandarin'],
 'also_known_as': ['Batman Begins 2'],
 'production_companies': ['Warner Bros.',
  'Legendary Entertainment',
  'Syncopy'],
 'filming_locations': ['Chicago, Illinois, USA'],
 'runtime': '2 hours 32 minutes'}

In [ ]:
batman_metadata.pop()

In [42]:
batman_metadata_['details']

{'release_date': ['July 18, 2008 (United States)'],
 'countries_of_origin': ['United States', 'United Kingdom'],
 'language': ['English', 'Mandarin'],
 'also_known_as': ['Batman Begins 2'],
 'production_companies': ['Warner Bros.',
  'Legendary Entertainment',
  'Syncopy'],
 'filming_locations': ['Chicago, Illinois, USA']}

In [21]:
from copy import deepcopy

batman_metadata_ = deepcopy(batman_metadata)
del batman_metadata_['poster']
batman_metadata_

{'original_title': 'The Dark Knight',
 'genres': ['Action', 'Crime', 'Drama'],
 'review_summary': {'user_reviews_num': '8.1K',
  'critic_review_num': '435',
  'metascore': '84'},
 'agg_rating': {'avg_rating': '9.0/10', 'num_votes': '2.6M'},
 'actors': {'Christian Bale': '/name/nm0000288?ref_=tt_cl_t_1',
  'Heath Ledger': '/name/nm0005132?ref_=tt_cl_t_2',
  'Aaron Eckhart': '/name/nm0001173?ref_=tt_cl_t_3',
  'Michael Caine': '/name/nm0000323?ref_=tt_cl_t_4',
  'Maggie Gyllenhaal': '/name/nm0350454?ref_=tt_cl_t_5',
  'Gary Oldman': '/name/nm0000198?ref_=tt_cl_t_6',
  'Morgan Freeman': '/name/nm0000151?ref_=tt_cl_t_7',
  'Monique Gabriela Curnen': '/name/nm1010931?ref_=tt_cl_t_8',
  'Ron Dean': '/name/nm0212939?ref_=tt_cl_t_9',
  'Cillian Murphy': '/name/nm0614165?ref_=tt_cl_t_10'},
 'imdb_recommendations': ['/title/tt1345836/?ref_=tt_sims_tt_t_1',
  '/title/tt0372784/?ref_=tt_sims_tt_t_2',
  '/title/tt1375666/?ref_=tt_sims_tt_t_3',
  '/title/tt0111161/?ref_=tt_sims_tt_t_4',
  '/title/tt

In [60]:
p = batman_metadata['poster']
type(p)

bytes

In [62]:
utils.save_img(p, 'the_dark_knight.jpeg', metadata_config, s3)

SaveResponse(is_successful=True, exception_msg='')

In [58]:
import boto3

# Let's use Amazon S3
s3 = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_access_key)

In [81]:
dynamodb = boto3.client(
    'dynamodb',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_access_key,
    region_name=metadata_config['dynamodb_region']
)

In [83]:
dynamodb.list_tables()

{'TableNames': ['movie-metadata'],
 'ResponseMetadata': {'RequestId': 'IDHSVA5468EMB5QEKEAHVNBP2BVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 03 Jun 2022 17:03:32 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '33',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'IDHSVA5468EMB5QEKEAHVNBP2BVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3100087865'},
  'RetryAttempts': 0}}

In [53]:
os.listdir('..')

['recsys.egg-info',
 'requirements.txt',
 'tests',
 'recsys',
 'docs',
 'README.md',
 'setup.py',
 'logs',
 '.gitignore',
 'scripts',
 '.git',
 '.vscode',
 'data',
 'notebooks']

In [169]:
batman_metadata_ = batman_metadata
_ = batman_metadata_.pop('poster')
batman_metadata_

{'original_title': None,
 'genres': ['Action', 'Crime', 'Drama'],
 'review_summary': {'user_reviews_num': '8.1K',
  'critic_review_num': '435',
  'metascore': '84'},
 'agg_rating': {'avg_rating': '9.0/10', 'num_votes': '2.6M'},
 'actors': {'Christian Bale': '/name/nm0000288?ref_=tt_cl_t_1',
  'Heath Ledger': '/name/nm0005132?ref_=tt_cl_t_2',
  'Aaron Eckhart': '/name/nm0001173?ref_=tt_cl_t_3',
  'Michael Caine': '/name/nm0000323?ref_=tt_cl_t_4',
  'Maggie Gyllenhaal': '/name/nm0350454?ref_=tt_cl_t_5',
  'Gary Oldman': '/name/nm0000198?ref_=tt_cl_t_6',
  'Morgan Freeman': '/name/nm0000151?ref_=tt_cl_t_7',
  'Monique Gabriela Curnen': '/name/nm1010931?ref_=tt_cl_t_8',
  'Ron Dean': '/name/nm0212939?ref_=tt_cl_t_9',
  'Cillian Murphy': '/name/nm0614165?ref_=tt_cl_t_10'},
 'imdb_recommendations': ['/title/tt1345836/?ref_=tt_sims_tt_t_1',
  '/title/tt0372784/?ref_=tt_sims_tt_t_2',
  '/title/tt1375666/?ref_=tt_sims_tt_t_3',
  '/title/tt0111161/?ref_=tt_sims_tt_t_4',
  '/title/tt7286456/?ref_

In [58]:
batman_metadata_

NameError: name 'batman_metadata_' is not defined

In [170]:
batman_metadata_['details']

{'release_date': ['July 18, 2008 (United States)'],
 'countries_of_origin': ['United States', 'United Kingdom'],
 'language': ['English', 'Mandarin'],
 'also_known_as': ['Batman Begins 2'],
 'production_companies': ['Warner Bros.',
  'Legendary Entertainment',
  'Syncopy'],
 'filming_locations': ['Chicago, Illinois, USA']}

In [175]:
metadata.collect_original_title(soup)

'The Dark Knight'

In [173]:
batman_metadata_

{'original_title': None,
 'genres': ['Action', 'Crime', 'Drama'],
 'review_summary': {'user_reviews_num': '8.1K',
  'critic_review_num': '435',
  'metascore': '84'},
 'agg_rating': {'avg_rating': '9.0/10', 'num_votes': '2.6M'},
 'actors': {'Christian Bale': '/name/nm0000288?ref_=tt_cl_t_1',
  'Heath Ledger': '/name/nm0005132?ref_=tt_cl_t_2',
  'Aaron Eckhart': '/name/nm0001173?ref_=tt_cl_t_3',
  'Michael Caine': '/name/nm0000323?ref_=tt_cl_t_4',
  'Maggie Gyllenhaal': '/name/nm0350454?ref_=tt_cl_t_5',
  'Gary Oldman': '/name/nm0000198?ref_=tt_cl_t_6',
  'Morgan Freeman': '/name/nm0000151?ref_=tt_cl_t_7',
  'Monique Gabriela Curnen': '/name/nm1010931?ref_=tt_cl_t_8',
  'Ron Dean': '/name/nm0212939?ref_=tt_cl_t_9',
  'Cillian Murphy': '/name/nm0614165?ref_=tt_cl_t_10'},
 'imdb_recommendations': ['/title/tt1345836/?ref_=tt_sims_tt_t_1',
  '/title/tt0372784/?ref_=tt_sims_tt_t_2',
  '/title/tt1375666/?ref_=tt_sims_tt_t_3',
  '/title/tt0111161/?ref_=tt_sims_tt_t_4',
  '/title/tt7286456/?ref_

In [65]:
dir_path = os.path.join('..', 'data', 'metadata')
dfs = []
for f in os.listdir(dir_path):
    if '.csv' not in f:
        continue
    file_path = os.path.join(dir_path, f)
    dfs.append(pd.read_csv(file_path))

ids = pd.concat(dfs)
ids.head()

,title_id,main_genre
0,/title/tt0468569/,crime
1,/title/tt0110912/,crime
2,/title/tt0068646/,crime
3,/title/tt1345836/,crime
4,/title/tt0114369/,crime


In [91]:
metadata_path = '../data/metadata/metadata.json'
ids.setto_json(metadata_path, orient='split')

In [97]:
URL_TEMPLATE = (
    'https://www.imdb.com/search/title/?title_type=feature&genres={}'
    '&sort=num_votes,desc&start={}&explore=genres&ref_=adv_nxt'
)
url = URL_TEMPLATE.format('action', 1)
url

'https://www.imdb.com/search/title/?title_type=feature&genres=action&sort=num_votes,desc&start=1&explore=genres&ref_=adv_nxt'

In [98]:
response = requests.get(url, headers={"Accept-Language": "en-US,en;q=0.5"})
ranking_soup = bs.BeautifulSoup(response.text, 'lxml')

In [144]:
j = {
    '/title/tt0468569/': {
        'main_genre': 'Action',
        'runtime': '2h'
    },
    '/title/tt1375666/': {
        'main_genre': 'Action',
        'runtime': '2h'
    }
}
j

{'/title/tt0468569/': {'main_genre': 'Action', 'runtime': '2h'},
 '/title/tt1375666/': {'main_genre': 'Action', 'runtime': '2h'}}

In [146]:
pd.DataFrame(j).T.to_json(metadata_path)

In [157]:
utils.write_json(j, metadata_path)

In [170]:
metadata_id = utils.read_json(metadata_path)
len(metadata_id)

3513

In [148]:
import pandas as pd
from recsys.core.data import CSVDataLoader
from recsys.etl.transformers import RawReviewsTransformer, RawDetailsTransformer
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [149]:
data_path = '../data/raw/details/1pct/action.csv'
loader = CSVDataLoader(data_path)
action_proc = RawDetailsTransformer(loader).transform()

In [150]:
action = loader.load_data()
action.head()

,original_title,review_summary,agg_rating,actors,imdb_recommendations,storyline,tagline,certificate,details,boxoffice,techspecs,title_id
0,Original title: The Dark Knight,"{'n_user_reviews': '7.7KUser reviews', 'n_crit...",9.0/102.5M,{'Christian Bale': '/name/nm0000288?ref_=tt_cl...,"['/title/tt1345836/?ref_=tt_sims_tt_t_1', '/ti...",Set within a year after the events of Batman B...,TaglinesWhy So Serious?,Certificate14+,"Release dateAugust 14, 2008 (Russia)Countries ...","Budget$185,000,000 (estimated)Gross US & Canad...",Runtime2 hours 32 minutesSound mixDolby Digita...,/title/tt0468569/
1,Original title: Inception,"{'n_user_reviews': '4.4KUser reviews', 'n_crit...",8.8/102.2M,{'Leonardo DiCaprio': '/name/nm0000138?ref_=tt...,"['/title/tt0816692/?ref_=tt_sims_tt_t_1', '/ti...","Dom Cobb is a skilled thief, the absolute best...",TaglinesYour mind is the scene of the crime,Certificate12+,"Release dateJuly 22, 2010 (Russia)Countries of...","Budget$160,000,000 (estimated)Gross US & Canad...",Runtime2 hours 28 minutesColorColorSound mixDo...,/title/tt1375666/
2,Original title: The Matrix,"{'n_user_reviews': '4.6KUser reviews', 'n_crit...",8.7/101.8M,{'Keanu Reeves': '/name/nm0000206?ref_=tt_cl_t...,"['/title/tt1375666/?ref_=tt_sims_tt_t_1', '/ti...",Thomas A. Anderson is a man living two lives. ...,TaglinesFree your mind,Certificate16+,"Release dateOctober 14, 1999 (Russia)Countries...","Budget$63,000,000 (estimated)Gross US & Canada...",Runtime2 hours 16 minutesColorColorSound mixDo...,/title/tt0133093/
3,Original title: The Lord of the Rings: The Fel...,"{'n_user_reviews': '5.5KUser reviews', 'n_crit...",8.8/101.7M,{'Elijah Wood': '/name/nm0000704?ref_=tt_cl_t_...,"['/title/tt0167261/?ref_=tt_sims_tt_t_1', '/ti...",An ancient Ring thought lost for centuries has...,TaglinesThe Legend Comes to Life,Certificate12+,"Release dateMarch 1, 2002 (Russia)Countries of...","Budget$93,000,000 (estimated)Gross US & Canada...",Runtime2 hours 58 minutesColorColorSound mixDT...,/title/tt0120737/
4,Original title: The Lord of the Rings: The Ret...,"{'n_user_reviews': '3.9KUser reviews', 'n_crit...",8.9/101.7M,{'Elijah Wood': '/name/nm0000704?ref_=tt_cl_t_...,"['/title/tt0167261/?ref_=tt_sims_tt_t_1', '/ti...",The final confrontation between the forces of ...,TaglinesThe eye of the enemy is moving.,Certificate12+,"Release dateJanuary 22, 2004 (Russia)Countries...","Budget$94,000,000 (estimated)Gross US & Canada...",Runtime3 hours 21 minutesColorColorSound mixDT...,/title/tt0167260/


In [151]:
from recsys.etl.functions import extract_movie_details

In [154]:
extract_movie_details(action.iloc[:2,:])

,original_title,review_summary,agg_rating,actors,imdb_recommendations,storyline,tagline,certificate,boxoffice,techspecs,title_id,release_date,country_of_origin,production_company
0,Original title: The Dark Knight,"{'n_user_reviews': '7.7KUser reviews', 'n_crit...",9.0/102.5M,{'Christian Bale': '/name/nm0000288?ref_=tt_cl...,"['/title/tt1345836/?ref_=tt_sims_tt_t_1', '/ti...",Set within a year after the events of Batman B...,TaglinesWhy So Serious?,Certificate14+,"Budget$185,000,000 (estimated)Gross US & Canad...",Runtime2 hours 32 minutesSound mixDolby Digita...,/title/tt0468569/,2008-08-14,"[United States, United Kingdom]","[Warner Bros., Legendary Entertainment, Syncopy]"
1,Original title: Inception,"{'n_user_reviews': '4.4KUser reviews', 'n_crit...",8.8/102.2M,{'Leonardo DiCaprio': '/name/nm0000138?ref_=tt...,"['/title/tt0816692/?ref_=tt_sims_tt_t_1', '/ti...","Dom Cobb is a skilled thief, the absolute best...",TaglinesYour mind is the scene of the crime,Certificate12+,"Budget$160,000,000 (estimated)Gross US & Canad...",Runtime2 hours 28 minutesColorColorSound mixDo...,/title/tt1375666/,2010-07-22,"[United Kingdom, United States]","[Warner Bros., Legendary Entertainment, Syncopy]"


In [8]:
data_path = '../data/raw/reviews/1pct/action.csv'
loader = CSVDataLoader(data_path)

In [10]:
action = loader.load_data()
action.head()

,id,text,rating,date,title,author,helpfulness
0,/title/tt0468569/,Best movie ever. Heath ledger's work is phenom...,10.0,12 January 2021,Perfect combo\n,/user/ur95396995/?ref_=tt_urv,\n 171 out of 185 found thi...
1,/title/tt0468569/,Totally one of the greatest movie titles ever ...,10.0,9 January 2021,The Dark Knight\n,/user/ur109215140/?ref_=tt_urv,\n 144 out of 158 found thi...
2,/title/tt0468569/,This movie is a work of art. The finest sequel...,10.0,17 February 2021,This town deserves a better class of criminal!\n,/user/ur129557514/?ref_=tt_urv,\n 50 out of 54 found this ...
3,/title/tt0468569/,"Confidently directed, dark, brooding, and pack...",10.0,12 February 2020,The Dark Knight\n,/user/ur87850731/?ref_=tt_urv,\n 404 out of 471 found thi...
4,/title/tt0468569/,It is just what you want for the best movie. G...,10.0,7 October 2019,MASTERPIECE\n,/user/ur108519953/?ref_=tt_urv,\n 217 out of 251 found thi...


In [27]:
action['text'][15]

'If someone else acted as Joker, I would give the movie 7-8 stars. The majority of people ended up loving the villain more than the hero, and that rarely happends in movies.Rest in peace Heath Ledger.'

In [28]:
reviews_processor = RawReviewsTransformer(loader)
a = reviews_processor.transform()

In [33]:
a['author'].str.split('?', expand=True)[0]

0         /user/ur95396995/
1        /user/ur109215140/
2        /user/ur129557514/
3         /user/ur87850731/
4        /user/ur108519953/
                ...        
66269     /user/ur53639379/
66270     /user/ur51801975/
66271     /user/ur36303263/
66272     /user/ur35145579/
66273     /user/ur44754847/
Name: 0, Length: 66274, dtype: object

In [36]:
a['title'].str.split('\n', expand=True)[0]

0                                          Perfect combo
1                                        The Dark Knight
2         This town deserves a better class of criminal!
3                                        The Dark Knight
4                                            MASTERPIECE
                              ...                       
66269                                 Unbelievably awful
66270                          How is this rated so low?
66271                     Good film despite bad reviews.
66272                 Pretty funny for a buddy cop movie
66273                                        Funny movie
Name: 0, Length: 66274, dtype: object